# Analyze runtimes

The referee asked about the total number of samples produced by the different pipelines.

In [2]:
import os
import psutil
p = psutil.Process()
p.cpu_affinity([0])
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
import jax.numpy as jnp
import jax
jax.config.update("jax_disable_jit", True)
import arviz

import h5py
import json

In [3]:
gwosc_path = "/home/thibeau.wouters/gw-datasets/GW190425/posterior_samples.h5"
jim_root_path = "/home/thibeau.wouters/TurboPE-BNS/real_events/"
bilby_root_path = "/home/thibeau.wouters/jim_pbilby_samples/older_bilby_version/"
paths_dict = {"GW170817_TaylorF2": {"jim": jim_root_path + "GW170817_TaylorF2/outdir/results_production.npz",
                    "bilby": bilby_root_path + "GW170817_TF2_with_tukey_fix_result.json"},
              
              "GW170817_NRTidalv2": {"jim": jim_root_path + "GW170817_NRTidalv2/outdir/results_production.npz",
                                     "bilby": bilby_root_path + "GW170817_IMRPhenomD_NRTidalv2_result.json",
                    },
              
              "GW190425_TaylorF2": {"jim": jim_root_path + "GW190425_TaylorF2/outdir_gwosc_data/results_production.npz",
                                    "bilby": bilby_root_path + "GW190425_GWOSC_data_result.json",
                    },
              
              "GW190425_NRTidalv2": {"jim": jim_root_path + "GW190425_NRTidalv2/outdir/results_production.npz",
                                     "bilby": bilby_root_path + "GW190425_NRTv2_GWOSC_data_result.json",
                    },
}

RUN_NAMES = list(paths_dict.keys())
JIM_VAR_NAMES = ['M_c', 'q', 's1_z', 's2_z', 'lambda_1', 'lambda_2', 'd_L', 't_c', 'phase_c', 'cos_iota', 'psi', 'ra', 'sin_dec']
BILBY_VAR_NAMES = ['chirp_mass', 'mass_ratio', 'spin_1z', 'spin_2z', 'lambda_1', 'lambda_2', 'luminosity_distance', 'phase', 'iota', 'psi', 'ra', 'dec']

## Jim

In [48]:
def get_nb_jim_samples(path: str, 
                       report_runtime_production: bool = False,
                       get_ess: bool = True):
    data = np.load(path)
    chains = data["chains"]
    a, b = np.shape(chains)[0], np.shape(chains)[1]
    
    # Also show the runtimes of the production loop
    if report_runtime_production:
        new_path = path.replace("results_production.npz", "runtime_production.txt")
        
        time = np.loadtxt(new_path)
        print("Runtime production:", time)
        
    # Compute the ESS
    data = np.load(path)
    chains = data["chains"]
    chains = np.reshape(chains, (int(a * b), 13))
    
    my_ess_list = []
    for values, name in zip(chains.T, JIM_VAR_NAMES):
        ess = arviz.ess(values)
        # print(f"ESS for {name} production: {ess}")
        my_ess_list.append(ess)
        
    avg_ess = int(np.floor(np.mean(my_ess_list)))
    
    return a * b, avg_ess

In [50]:
def my_avg(values):
    """Take the average but floor and int it"""
    
    return int(np.floor(np.mean(list(values))))

In [52]:
total_nb_samples_jim = {}
ess_jim = {}

for run_name in RUN_NAMES:
    total, ess = get_nb_jim_samples(paths_dict[run_name]["jim"])
    total_nb_samples_jim[run_name] = total
    ess_jim[run_name] = ess
    
print("Total samples for Jim")
print(total_nb_samples_jim)

print("ESS for Jim")
print(ess_jim)

avg_ess_jim = my_avg(list(ess_jim.values()))
print("Average ESS for Jim")
print(avg_ess_jim)

Total samples for Jim
{'GW170817_TaylorF2': 220000, 'GW170817_NRTidalv2': 220000, 'GW190425_TaylorF2': 600000, 'GW190425_NRTidalv2': 600000}
ESS for Jim
{'GW170817_TaylorF2': 18903, 'GW170817_NRTidalv2': 16488, 'GW190425_TaylorF2': 106992, 'GW190425_NRTidalv2': 33828}
Average ESS for Jim
44052


All the runtimes are below 1 minute -- love jax <3

## Bilby

In [53]:
def get_nb_bilby_samples(path: str):
    
    with open(path, "r") as f:
        data = json.load(f)
    posterior = data["posterior"]["content"]
    mc = posterior["chirp_mass"] # just taking the chirp mass samples as an example
    n = len(mc)
    
    # Compute the ESS
    my_ess_list = []
    for name in BILBY_VAR_NAMES:
        values = posterior[name]
        
        # NOTE: sometimes the json is broken and there are dicts instead of floats
        if isinstance(values[0], dict):
            new_values = [item['content'] for item in values]
            values = new_values

        values = np.array(values)

        ess = arviz.ess(values)
        my_ess_list.append(ess)
        
    avg_ess = int(np.floor(np.mean(my_ess_list)))
        
    return n, avg_ess

### pBilby

In [54]:
total_nb_samples_bilby = {}
ess_bilby = {}

for run_name in RUN_NAMES:
    path = paths_dict[run_name]["bilby"]
    total, ess = get_nb_bilby_samples(path)
    
    total_nb_samples_bilby[run_name] = total
    ess_bilby[run_name] = ess
    
print("Total samples for Bilby")
print(total_nb_samples_bilby)

print("ESS for Bilby")
print(ess_bilby)

avg_ess_bilby = my_avg(list(ess_bilby.values()))
print("Average ESS for Bilby")
print(avg_ess_bilby)

Total samples for Bilby
{'GW170817_TaylorF2': 44782, 'GW170817_NRTidalv2': 45365, 'GW190425_TaylorF2': 30782, 'GW190425_NRTidalv2': 30277}
ESS for Bilby
{'GW170817_TaylorF2': 3982, 'GW170817_NRTidalv2': 4177, 'GW190425_TaylorF2': 5389, 'GW190425_NRTidalv2': 4709}
Average ESS for Bilby
4564


### Relative binning-Bilby

In [55]:
rb_paths = ["../RB/gw170817_relbin_TaylorF2_result.hdf5", 
            "../RB/gw170817_relbin_result.hdf5",
            "../RB/gw190425_relbin_TaylorF2_result.hdf5",
            "../RB/gw190425_relbin_result.hdf5"]

def get_nb_rb_bilby_samples(path: str):
    
    with h5py.File(path, "r") as f:
        posterior = f["posterior"]
        mc = posterior["chirp_mass"][()] # just taking the chirp mass samples as an example
        n = len(mc)
        
        my_ess_list = []
        for name in BILBY_VAR_NAMES:
            values = posterior[name][()]
            
            # NOTE: sometimes the json is broken and there are dicts instead of floats
            if isinstance(values[0], dict):
                new_values = [item['content'] for item in values]
                values = new_values

            values = np.array(values)

            ess = arviz.ess(values)
            # print(f"ESS for {name} production: {ess}")
            my_ess_list.append(ess)
                
        avg_ess = int(np.floor(np.mean(my_ess_list)))
    
    return n, avg_ess

In [56]:
total_nb_samples_rb_bilby = {}
ess_rb_bilby = {}

for path, run_name in zip(rb_paths, RUN_NAMES):
    print(run_name)
    total, ess = get_nb_rb_bilby_samples(path)

    total_nb_samples_rb_bilby[run_name] = total
    ess_rb_bilby[run_name] = ess
    
print("Total samples for RB Bilby")
print(total_nb_samples_rb_bilby)

print("ESS for RB Bilby")
print(ess_rb_bilby)

avg_ess_rb_bilby = my_avg(list(ess_rb_bilby.values()))
print("Average ESS for RB Bilby")
print(avg_ess_rb_bilby)

GW170817_TaylorF2
GW170817_NRTidalv2
GW190425_TaylorF2
GW190425_NRTidalv2
Total samples for RB Bilby
{'GW170817_TaylorF2': 5258, 'GW170817_NRTidalv2': 5172, 'GW190425_TaylorF2': 6743, 'GW190425_NRTidalv2': 5172}
ESS for RB Bilby
{'GW170817_TaylorF2': 3274, 'GW170817_NRTidalv2': 3837, 'GW190425_TaylorF2': 3661, 'GW190425_NRTidalv2': 3837}
Average ESS for RB Bilby
3652


### ROQ-Bilby

In [43]:
roq_paths = ["/home/thibeau.wouters/TurboPE-BNS/ROQ/gw170817_ROQ_result.hdf5",
             "/home/thibeau.wouters/TurboPE-BNS/ROQ/gw190425_ROQ_result.hdf5"]

total_nb_samples_roq_bilby = {}
ess_roq_bilby = {}

for path, run_name in zip(rb_paths, RUN_NAMES):
    print(run_name)
    total, ess = get_nb_rb_bilby_samples(path)

    total_nb_samples_rb_bilby[run_name] = total
    ess_rb_bilby[run_name] = ess
    
print("Total samples for RB Bilby")
print(total_nb_samples_rb_bilby)

print("ESS for RB Bilby")
print(ess_rb_bilby)

avg_ess_rb_bilby = my_avg(list(ess_rb_bilby.values()))
print("Average ESS for RB Bilby")
print(avg_ess_rb_bilby)

GW170817_TaylorF2
Average ESS: 2685
GW170817_NRTidalv2
Average ESS: 2414


{'GW170817_TaylorF2': 6783, 'GW170817_NRTidalv2': 6588}